### Preparation

In [ ]:
CHATGPT_KEY = "sk-..."
CLAUDE_KEY = "sk-ant-..."
# GEMINI_KEY = "..."

In [ ]:
# !pip install langchain
# !pip install langchain-openai
# !pip install langchain-anthropic
# !pip install langchain-google-genai

In [ ]:
# from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

# gemini = ChatGoogleGenerativeAI(model = "gemini-pro", google_api_key = GEMINI_KEY)
claude = ChatAnthropic(model_name = "claude-3-haiku-20240307", anthropic_api_key = CLAUDE_KEY)
chatgpt = ChatOpenAI(model = "gpt-3.5-turbo", openai_api_key = CHATGPT_KEY)

### Match Functions

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

basic = HTTPBasicAuth('admin', 'ethanPassAdmin')
# server_url = "http://localhost:3000" # For local server
server_url = "https://gen-ai-tut-backend.up.railway.app"

def handle_response(response: requests.Response) -> dict|None:
        try:
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as err:
            print(f"Error: {response.status_code}. {err}")
            return None

In [ ]:
from api.models import Match

def get_match_list():
    response = requests.get(f'{server_url}/match/list', auth=basic)
    res = handle_response(response)
    return res

def get_finished_match(matches:list, processing:list) -> tuple:
    try:
        for i, match in enumerate(matches):
            if match.match_status == "FINISHED" and match.id not in processing:
                return i, match.id
        return -1, -1
    except:
        return -1, -1    

def get_match_info(match_id) -> dict:
    response = requests.get(f'{server_url}/match/admin/{match_id}' , auth=basic)
    return response.json()

def get_history_messages(match_info: Match) -> list:
    return [msg["text"] for msg in match_info["historyMsgs"]]

In [ ]:

def get_basic_info(match: Match, match_info: dict) -> tuple:
    topics = {"1": "支不支持韓國醫生罷工", "2": "支不支持必勝客推出草仔粿烏龜披薩", "3": "使用AI協助創作應不應該註明"}
    topic_id = match.topic_id
    topic = topics[str(topic_id)]
    first_player_id = match_info["firstPlayerId"]
    player_ids = [ player["playerId"] for player in match_info["players"]]
    
    if player_ids[0] == first_player_id:
        second_player_id = player_ids[1]
    else:
        second_player_id = player_ids[0]
    return topic, first_player_id, second_player_id



### Evaluator

In [ ]:
class Evaluator():
  def __init__(self, topic, message):
    self.topic = topic
    self.message = message
    self.verify_num = 1
    self.message_prefix = [
      "正方論述:",
      "反方論述:",
      "正方反駁:",
      "反方反駁:"
    ]
    self.message[-1]= self.message[-1].split("Human")[0]
    self.concat_message = [self.message_prefix[i] + self.message[i] for i in range(len(self.message))]
    self.prompt_template_first = """
你是一個專業的評論裁判，本次辯論題目是「<<topic>>」，請根據以下標準判斷正方獲勝或是反方獲勝。

勝負判斷標準:
1. 邏輯性和說服力： 參賽者的論點應該有邏輯性且具有說服力。他們應該能夠清晰地陳述他們的觀點，並提供具體的證據和理由來支持這些觀點。

2. 構建論據： 參賽者應該能夠有效地構建論據來支持他們的立場。這可能包括引用研究、統計數據、專家見解等，以加強他們的論點。

3. 反駁和反證： 參賽者應該能夠有效地反駁對手的論點，並提供反證來支持自己的立場。這展示了他們的辯論技巧和對話能力。

4. 語言和表達： 參賽者應該使用適當的語言和表達方式來溝通他們的想法。清晰、有條理的語言和良好的口頭表達能力對於贏得辯論至關重要。

5. 尊重和態度： 參賽者應該表現出尊重對手和裁判的態度。他們應該避免使用攻擊性言語或不當行為，並且應該遵守辯論比賽的規則和標準。

<<message>>
<<message>>
<<message>>
<<message>>
    """
    self.prompt_template_second = """
完全相信上面你的分析，判斷正方獲勝或是反方獲勝，只要輸出"正方獲勝"或"反方獲勝"
    """
  def get_prompt(self, concat_message):
    prompt = self.prompt_template_first
    prompt = prompt.replace("<<topic>>", self.topic, 1)
    for i in concat_message:
      prompt = prompt.replace("<<message>>", i, 1)
    return prompt

  def get_referee_result(self, prompt, llm):
    message = [{"role": "user", "content": prompt}]
    comment = llm.invoke(message).content
    message.append({"role": "assistant", "content": comment})
    message.append({"role": "user", "content": self.prompt_template_second})
    result = llm.invoke(message).content
    if "正方獲勝" in result:
      return "正方獲勝"
    elif "反方獲勝" in result:
      return "反方獲勝"
    return result
  
  def get_result(self):
    result = []
    first = True
    for llm in [claude, chatgpt]:
      for i in range(4):
        if i % 2 == 0:
          self.concat_message[0], self.concat_message[1] = self.concat_message[1], self.concat_message[0]
        else:
          self.concat_message[2], self.concat_message[3] = self.concat_message[3], self.concat_message[2]
        prompt = self.get_prompt(self.concat_message)
        
        for i in range(self.verify_num):
          res = self.get_referee_result(prompt, llm)
          if first:
            result.append(res)
            first = False
          result.append(res)
    
    print(result)
    return {"正方獲勝": result.count("正方獲勝"), "反方獲勝": result.count("反方獲勝"), "平手": 9 * self.verify_num - result.count("正方獲勝") - result.count("反方獲勝")}

### Result

In [ ]:
def find_winner(result, first_player_id, second_player_id):
    winner_id = ""
    if result["正方獲勝"] > result["反方獲勝"]:
        print(f"勝者為正方 id: {first_player_id}")
        winner_id = first_player_id
    elif result["反方獲勝"] > result["正方獲勝"]:
        print(f"勝者為反方 id: {second_player_id}")
        winner_id = second_player_id
    else:
        print("平手")
        winner_id = first_player_id
    return winner_id

def submit_result(finished_match_id, result, first_player_id, second_player_id):
    url = f'{server_url}/result/submit'
    winner_id = find_winner(result, first_player_id, second_player_id)
    data = {
        "winnerId": winner_id,
        "comment": "Good. very good",
        "matchId": finished_match_id,
        "points": [{"userId": first_player_id, "points": result["正方獲勝"]}, {"userId": second_player_id, "points": result["反方獲勝"]}]
    }
    response = requests.post(url, json=data, auth=basic)
    return response


### Loop

In [ ]:

def evaluate(finished_match_id, processing_match):
    print(f"evaluating match: {finished_match_id}")
    match_info = get_match_info(finished_match_id)
    msgs = get_history_messages(match_info)
    topic, first_player_id, second_player_id = get_basic_info(processing_match,match_info)
    result = Evaluator(topic, msgs).get_result()
    response = submit_result(finished_match_id, result, first_player_id, second_player_id)
    res = handle_response(response)
    print(f"Match {finished_match_id} evaluation result: {result}")


In [ ]:
import time
import threading
from queue import Queue

processing = []
match_queue = Queue()

def worker():
    while True:
        # print(match_queue.qsize())
        if match_queue.empty():
            time.sleep(5)
            continue
        finished_match_id, match = match_queue.get()
        evaluate(finished_match_id, match)
        match_queue.task_done()

threads = []
num_threads = 4  # 你可以根據需要調整線程數量

for _ in range(num_threads):
    t = threading.Thread(target=worker)
    t.start()
    threads.append(t)

while True:
    matches = get_match_list()
    matches = [Match.from_json(match) for match in matches]
    index, finished_match_id = get_finished_match(matches,processing)
    if index == -1:
        time.sleep(5)
        continue
    processing.append(finished_match_id)
    match_queue.put((finished_match_id, matches[index]))
    time.sleep(5)



In [ ]:
match_queue.join()
for _ in range(num_threads):
    match_queue.put(None)
for t in threads:
    t.join()